# Generate BCPP requisition and aliquot datasets

This notebook creates two files:
1. `lab_requisition_{timestamp}.csv`
2. `lab_aliquot_{timestamp}.csv`

## Usage:

* Run the notebook server in a valid BCPP VENV
* Start the `jupyter notebook` server in the root of the main project (e.g. bcpp/) or, if the main project is part of the VENV, from any folder with this notebook file.
* the database connection is that configured in `bcpp.settings.py`. To change this, create your own `settings.py` and specify with `manage.py`'s `--settings` option.
* start the server using:

        python manage.py shell_plus --notebook --settings=bcpp.settings
        
## Tunnel to DB

Open a tunnel to the live DB:

    ssh -f <user>@<server> -L5002:localhost:3306 -N

Change the mysql settings (e.g. in your `/etc/bcpp/mysql.conf`):

    port: 5002
    name: <database name>
    user: <readonly user>
    password: <password>
    


In [ ]:
import numpy as np
import os
import pandas as pd
import sys


from arrow import Arrow
from bcpp_community import communities
from bcpp_status.models import StatusHistory
from bcpp_subject.models import SubjectVisit, SubjectConsent
from copy import copy
from datetime import datetime
from django.db import connection
from edc_constants.constants import YES, NO, NEG, UNK
from edc_pdutils.model_to_dataframe import ModelToDataframe, SubjectModelToDataframe, Helper
from pprint import pprint
from edc_base.constants import DEFAULT_BASE_FIELDS
from edc_lab.model_mixins.requisition import RequisitionStatusMixin

date_format = '%Y-%m-%d %H:%M:%S.%f'
export_date_format = '%Y-%m-%d'
local_tz = 'Africa/Gaborone'
yes_no = {True: YES, False: NO}
timestamp = datetime.today().strftime('%Y%m%d%H%M%S')

export_path = '~/Documents/bcpp/moyo/'
export_path = os.path.expanduser(export_path)

delimiter = ',' # '|'
start_date = datetime(2013, 10, 1)



In [ ]:
class StatusHistoryModelToDataframe(ModelToDataframe):
    columns = {fld.name: fld.name for fld in StatusHistory._meta.get_fields()}    

In [ ]:
helper = Helper()

In [ ]:
df_subjects = SubjectModelToDataframe(model='bcpp_subject.subjectconsent').dataframe
df_subjects = df_subjects.drop_duplicates()
# df_subjects.info()

In [ ]:
df_subjects.head()

In [ ]:
# start with subject visit model
cols = ['subject_identifier', 'report_date', 'visit_code', 'consent_version', 'survey', 'household_member_id', ]
df = ModelToDataframe(model='bcpp_subject.subjectvisit', drop_sys_columns=True).dataframe
df = df.rename(columns={'id': 'subject_visit_id'})
df = df.set_index('subject_visit_id')
df['report_date'] = helper.to_local_datetime(df['report_datetime'])
df['report_date'] = df['report_date'].dt.normalize()
df = df[cols]
df_original = df.copy()

# only keep records after start_date
# df = df[df['report_date'] >= start_date]

In [ ]:
# subjectconsent
model = 'bcpp_subject.subjectconsent'
cols = ['subject_identifier', 'gender', 'dob', 'consent_datetime', 'version']
df_consent = SubjectModelToDataframe(model=model, columns=cols).dataframe
df_consent['consent_datetime'] = helper.to_local_datetime(df_consent['consent_datetime'])
df_consent['dob'] = helper.date_to_local_datetime(df_consent['dob'])

# remove subject identifier as UUID
df_consent = df_consent[df_consent['subject_identifier'].str.len() != 32]

# drop duplicates (because of versions)
df_consent = df_consent.sort_values(['subject_identifier'])
df_consent = df_consent.drop_duplicates(['subject_identifier'], keep='first')
df_consent = df_consent[['subject_identifier', 'gender', 'dob']]

In [ ]:
# merge subjectconsent to main df
df = df.reset_index()
df = pd.merge(df, df_consent, on='subject_identifier', how='left')

In [ ]:
# set index for joins with CRFs
df = df.set_index('subject_visit_id')

In [ ]:
# subjectrequisition
status_cols = [f.name for f in RequisitionStatusMixin._meta.get_fields()
               if f not in ['processed_datetime', 'packed_datetime', 'shipped_datetime']]
cols = ['requisition_identifier', 'panel_name', 'requisition_datetime', 'is_drawn', 'reason_not_drawn',
        'drawn_datetime', 'specimen_type', 'study_site', 'study_site_name']
cols.extend([f.name for f in RequisitionStatusMixin._meta.get_fields()])
df_crf = helper.get_crf_dataframe(
    model='bcpp_subject.subjectrequisition', cols=cols)
df_crf['requisition_datetime'] = helper.to_local_datetime(df_crf['requisition_datetime'])
df_crf['drawn_datetime'] = helper.to_local_datetime(df_crf['drawn_datetime'])
df_crf['received_datetime'] = helper.to_local_datetime(df_crf['received_datetime'])
df_crf['received'] = df_crf['received'].map(yes_no, na_action='ignore')
df_crf['processed'] = df_crf['processed'].map(yes_no, na_action='ignore')
df_crf['packed'] = df_crf['packed'].map(yes_no, na_action='ignore')
df_crf['shipped'] = df_crf['shipped'].map(yes_no, na_action='ignore')

# join to main df
df = df.join(df_crf)

# show grouping
# df.groupby('circumcised').size()

In [ ]:
df.head()

In [ ]:
df.groupby('panel_name').size()

In [ ]:
# 'bcpp_status.statushistory'
df_status = StatusHistoryModelToDataframe(model='bcpp_status.statushistory').dataframe
cols = ['subject_identifier', 'status_date', 'timepoint', 'final_hiv_status', 'final_hiv_status_date', 'final_arv_status', ]
df_status = df_status[cols]
df_status = df_status.rename(columns={'status_date': 'report_date', 'timepoint': 'visit_code'})
df_status['report_date'] = helper.date_to_local_datetime(df_status['report_date'])
df_status['final_hiv_status_date'] = helper.date_to_local_datetime(df_status['final_hiv_status_date'])
df_status = df_status.groupby(['subject_identifier', 'report_date']).last()
df_status = df_status.reset_index()
df_status = df_status[['subject_identifier',  'report_date', 'visit_code', 'final_hiv_status', 'final_hiv_status_date', 'final_arv_status']]
df = pd.merge(df, df_status, on=['subject_identifier', 'report_date', 'visit_code'], how='left')


In [ ]:
# remove invalid sites
df = df[-df['study_site_name'].isin(['bhp', 'test_community'])]
df = df.drop(['specimen_type', 'household_member_id', 'final_hiv_status', 'final_hiv_status_date', 'final_arv_status'], axis=1)

In [ ]:
# fix reason_not_drawn
is_blank = (df.reason_not_drawn.notnull()) & (df.is_drawn == YES)
df.loc[is_blank, 'reason_not_drawn'] = np.nan

In [ ]:
df.head()

In [ ]:
# export requisitions to CSV as a single file
path = os.path.join(export_path, f'lab_requisitions_{timestamp}.csv')
df.to_csv(path, index=True, date_format=export_date_format, sep=delimiter)
sys.stdout.write(f'* {path}\n')

In [ ]:
# df.groupby('study_site_name').size()
# df.groupby('is_drawn').size()
# df.groupby('panel_name').size()
# df.groupby('visit_code').size()
# df.groupby('survey').size()
# df.groupby('gender').size()
# print(df['report_date'].min())
# print(df['report_date'].max())
# print(df['drawn_datetime'].min())
# print(df['drawn_datetime'].max())
# print(df['requisition_datetime'].min())
# print(df['requisition_datetime'].max())
# print(df['dob'].min())
# print(df['dob'].max())


In [ ]:
model = 'edc_lab.aliquot'
cols = None
df_aliquot = ModelToDataframe(model=model).dataframe
df_aliquot['aliquot_datetime'] = helper.to_local_datetime(df_aliquot['aliquot_datetime'])

In [ ]:
# df_aliquot.groupby('hostname_created').size()

In [ ]:
# drop sys and other unwanted columns
base_cols = copy(DEFAULT_BASE_FIELDS)
base_cols.pop(base_cols.index('hostname_created'))
base_cols
# columns = DEFAULT_BASE_FIELDS + ['slug', 'comment', 'shipped']
# df_aliquot = df_aliquot.drop(columns, axis=1)

In [ ]:
df_aliquot['is_primary'] = df_aliquot['is_primary'].map(yes_no, na_action='ignore')
df_aliquot['medium'] = df_aliquot['medium'].str.lower()

In [ ]:
# add column to indicate missing requisitions
cols = ['requisition_identifier',
        'panel_name', 'requisition_datetime',
        'gender', 'dob', 'study_site', 'study_site_name']
df1 = pd.merge(df_aliquot, df[cols],
               on='requisition_identifier', how='left')
df_aliquot['missing_requisition'] = df1['panel_name'].isnull()
df_aliquot['is_primary'] = df_aliquot['is_primary'].map(yes_no, na_action='ignore')
df_aliquot.head()
df_aliquot.info()

In [ ]:
# export aliquots to CSV as a single file
path = path = os.path.join(export_path, f'lab_aliquots_{timestamp}.csv')
df_aliquot.to_csv(path, index=True, date_format=export_date_format, sep=delimiter)
sys.stdout.write(f'* {path}\n')

In [ ]:
# show some stats
print(df_aliquot.groupby('medium').size())
print('---')

print(df_aliquot.groupby('aliquot_type').size())
print('---')

print(df_aliquot.groupby('alpha_code').size())
print('---')

print(df_aliquot.groupby('numeric_code').size())
print('---')

print(df_aliquot.groupby('condition').size())
print('---')

print(df_aliquot.groupby('missing_requisition').size())
print('---')

### Missing requisitions

In [ ]:
df1 = df_aliquot[df_aliquot['missing_requisition']]
df1 = df1.sort_values(['aliquot_datetime'])

In [ ]:
df1.groupby(['hostname_created', 'device_created']).size()

In [ ]:
df1[['requisition_identifier']].drop_duplicates()


In [ ]:
cols = ['subject_identifier', 'aliquot_identifier', 'requisition_identifier', 'created',
        'device_created', 'hostname_created']
df1[cols].to_csv(os.path.join(export_path, 'missing_requisition_{timestamp}.csv'))